In [37]:
import yfinance as yf
import pandas as pd
import numpy as np

import datetime 

### Interval : Day

In [38]:
apple= yf.Ticker("aapl")
sp = yf.Ticker("^GSPC")
apple_price = apple.history(start="2018-01-01", end="2022-12-31", interval="1d")
sp_price = sp.history(start="2018-01-01", end="2022-12-31", interval="1d")

In [39]:
columns_drop = ['Dividends', 'High', 'Low', 'Volume', 'Stock Splits', 'Open']
apple_price = apple_price.drop(columns=columns_drop)
sp_price = sp_price.drop(columns=columns_drop)

In [40]:
apple_price = pd.merge(apple_price, sp_price, on="Date", how="left").reset_index()

In [41]:
pd.concat([pd.Series([1,2,3]), apple_price])

,0,Close_x,Close_y,Date
0,1.0,NaN,NaN,NaT
1,2.0,NaN,NaN,NaT
2,3.0,NaN,NaN,NaT
0,NaN,40.888069,2695.810059,2018-01-02 00:00:00-05:00
1,NaN,40.880939,2713.060059,2018-01-03 00:00:00-05:00
...,...,...,...,...
1254,NaN,131.658981,3844.820068,2022-12-23 00:00:00-05:00
1255,NaN,129.831772,3829.250000,2022-12-27 00:00:00-05:00
1256,NaN,125.847855,3783.219971,2022-12-28 00:00:00-05:00
1257,NaN,129.412415,3849.280029,2022-12-29 00:00:00-05:00


In [42]:
def addMissingDate(data): 
  missing_rows = pd.DataFrame(columns= ["Date", 'Close_x', 'Close_y'])
  first_date = data.iloc[0].Date
  last_date = data.iloc[len(data) - 1].Date
  if (first_date.weekday() != 0):
    for i in range(first_date.weekday() -1, -1, -1):
      first = data.iloc[0]
      missing_rows.loc[len(missing_rows)]= [first_date - datetime.timedelta(days=i), first.Close_x, first.Close_y ]

  if (last_date.weekday() != 4):
    for i in range(1, 4 - last_date.weekday() + 1):
      last = data.iloc[len(data) - 1]
      missing_rows.loc[len(missing_rows)]= [last_date + datetime.timedelta(days=i), last.Close_x, last.Close_y ]
  
  res = pd.concat([missing_rows, data]).sort_values('Date')

  missing_rows = pd.DataFrame(columns= ["Date", 'Close_x', 'Close_y'])
  prev = res.iloc[0].Date
  counter = 0
  for i in range(len(res)):
    cur_row = res.iloc[i]
    day_diff = (cur_row.Date - prev).days
    
    if ((cur_row.Date.weekday() == 0 & prev.weekday() == 4) or (day_diff == 1)):
      prev = cur_row.Date
      continue

    elif ((day_diff > 4) & (cur_row.Date.weekday() > 0) & (prev.weekday() < 4)):
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing ending and start")
      for j in range(1, (5 - prev.weekday())):
        missing_rows.loc[len(missing_rows)]= [prev + datetime.timedelta(days=j), res.iloc[i-1].Close_x, res.iloc[i-1].Close_y ]
        counter += 1

      for k in range(0, cur_row.Date.weekday()):
        missing_rows.loc[len(missing_rows)]= [cur_row.Date - datetime.timedelta(days=k+1), cur_row.Close_x, cur_row.Close_y ]
        counter += 1
    
    elif ((cur_row.Date.weekday() > 0) & (prev.weekday() == 4)):
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing start")
      for k in range(0, cur_row.Date.weekday()):
        missing_rows.loc[len(missing_rows)]= [cur_row.Date - datetime.timedelta(days=k+1), cur_row.Close_x, cur_row.Close_y ]
        counter += 1
    
    elif ((cur_row.Date.weekday() == 0) & (prev.weekday() != 4)):
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing ending")
      for j in range(1, (5 - prev.weekday())):
        missing_rows.loc[len(missing_rows)]= [prev + datetime.timedelta(days=j), res.iloc[i-1].Close_x, res.iloc[i-1].Close_y ]
        counter += 1
    
    
    
       
    elif ((day_diff > 1) & (cur_row.Date.weekday() > prev.weekday()) ) :
      # print(cur_row.Date.weekday(), prev.weekday(), day_diff)
      # print("missing mid")
      for j in range(1, day_diff):
        missing_rows.loc[len(missing_rows)]= [prev + datetime.timedelta(days=j), res.iloc[i-1].Close_x, res.iloc[i-1].Close_y ]
        counter += 1

    prev = cur_row.Date
  print(counter)
  res = pd.concat([missing_rows, res]).sort_values('Date').reset_index().drop(columns="index")

  return res


In [43]:
addMissingDate(apple_price)

45


,Date,Close_x,Close_y
0,2018-01-02 00:00:00-05:00,40.888069,2695.810059
1,2018-01-02 00:00:00-05:00,40.888069,2695.810059
2,2018-01-03 00:00:00-05:00,40.880939,2713.060059
3,2018-01-04 00:00:00-05:00,41.070827,2723.989990
4,2018-01-05 00:00:00-05:00,41.538437,2743.149902
...,...,...,...
1300,2022-12-26 00:00:00-05:00,129.831772,3829.250000
1301,2022-12-27 00:00:00-05:00,129.831772,3829.250000
1302,2022-12-28 00:00:00-05:00,125.847855,3783.219971
1303,2022-12-29 00:00:00-05:00,129.412415,3849.280029


### Interval : Week


In [44]:
apple= yf.Ticker("aapl")
sp = yf.Ticker("^GSPC")
applep = apple.history(start="2019-01-01", end="2022-12-31", interval="1wk")
spp = sp.history(start="2019-01-01", end="2022-12-31", interval="1wk")

In [45]:
columns_drop = ['Dividends', 'Volume', 'Stock Splits', 'Open']
columns_drop_sp = ['Dividends', 'Volume', 'Stock Splits', 'Open', 'High', 'Low']
applep = applep.drop(columns=columns_drop)
spp = spp.drop(columns=columns_drop_sp)
applep = pd.merge(applep, spp, on="Date", how="left").reset_index()

In [46]:
applep

,Date,High,Low,Close_x,Close_y
0,2019-01-01 00:00:00-05:00,38.271110,34.211504,35.640194,2549.689941
1,2019-01-08 00:00:00-05:00,37.230311,35.782346,36.138916,2582.610107
2,2019-01-15 00:00:00-05:00,38.037402,36.150950,37.782021,2670.709961
3,2019-01-22 00:00:00-05:00,38.097649,36.548492,37.656754,2643.850098
4,2019-01-29 00:00:00-05:00,41.357375,37.129121,41.258595,2724.870117
...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,150.689917,140.136039,146.406464,3998.840088
205,2022-12-06 00:00:00-05:00,147.075443,139.786569,144.269730,3990.560059
206,2022-12-13 00:00:00-05:00,149.741373,131.119811,132.168198,3817.659912
207,2022-12-20 00:00:00-05:00,136.601432,129.442364,131.658981,3844.820068


In [47]:
# import requests
# import json

# # set the API endpoint and parameters
# symbol = 'AAPL'
# api_key = 'U9NBHARV1LPZMLN0'
# endpoint = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={api_key}'

# # send a request to the API endpoint
# response = requests.get(endpoint)

# # parse the response JSON data
# data = json.loads(response.content)

# # extract the quarterly earnings data
# earnings = data['quarterlyEarnings']

# # convert the earnings data to a pandas dataframe
# df = pd.DataFrame.from_dict(earnings)

# # set the date column as the index and sort the dataframe by date
# df = df.set_index('reportedDate').sort_index()

# # filter the dataframe based on start and end dates
# start_date = '2018-04-01'
# end_date = '2023-03-27'
# df = df.loc[start_date:end_date]

# # calculate the trailing twelve months (TTM) earnings
# ttm_earnings = []
# for i in range(len(df)):
#     if i < 3:
#         ttm_earnings.append(None)
#     else:
#         first = float(df.iloc[i-3]['reportedEPS'])
#         second = float(df.iloc[i-2]['reportedEPS'])
#         third = float(df.iloc[i-1]['reportedEPS'])
#         fourth = float(df.iloc[i]['reportedEPS'])
#         ttm_earnings.append(first+second+third+fourth)

# df['ttmEarnings'] = ttm_earnings

# # calculate the PE ratio as the market price per share divided by TTM earnings
# # df['peRatio'] = df['close'] / df['ttmEarnings']

# # print the dataframe
# print(df)

In [48]:
import requests
import json

# set the API endpoint and parameters
symbol = 'AAPL'
api_key = 'U9NBHARV1LPZMLN0'

endpoint_earning = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={api_key}'
endpoint_ics = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={api_key}'
endpoint_bs = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={api_key}'

# send a request to the API endpoint
response_e = requests.get(endpoint_earning)
response_ics = requests.get(endpoint_ics)
response_bs = requests.get(endpoint_bs)

# parse the response JSON data
data_e = json.loads(response_e.content)
data_ics = json.loads(response_ics.content)
data_bs = json.loads(response_bs.content)

# cols needed 
cols = {
    'bs': ['cashAndCashEquivalentsAtCarryingValue', 'shortLongTermDebtTotal','commonStockSharesOutstanding', 'totalLiabilities', 'totalShareholderEquity'],
    'is': ['ebitda', 'netIncome'],
    'e' : ['reportedEPS']
    }


# extract the quarterly earnings data
earnings = data_e['quarterlyEarnings']
ics = data_ics['quarterlyReports']
bs = data_bs['quarterlyReports']

# convert the earnings data to a pandas dataframe
earnings_df = pd.DataFrame.from_dict(earnings)
ics_df = pd.DataFrame.from_dict(ics)
bs_df = pd.DataFrame.from_dict(bs)

# set the date column as the index and sort the dataframe by date
earnings_df = earnings_df.set_index('reportedDate').sort_index()
ics_df = ics_df.set_index('fiscalDateEnding').sort_index()
bs_df = bs_df.set_index('fiscalDateEnding').sort_index()

# filter the dataframe based on start and end dates
start_date = '2018-04-01'
end_date = '2023-03-27'

start_date_earning = '2019-01-29'
end_date_earning = '2022-10-27'
earnings_df = earnings_df.loc[start_date_earning:end_date_earning][cols['e']]
ics_df = ics_df.loc[start_date:end_date][cols['is']]
bs_df = bs_df.loc[start_date:end_date][cols['bs']]

df = ics_df.merge(bs_df, on='fiscalDateEnding')

for col in ['ebitda','commonStockSharesOutstanding', 'totalLiabilities', 'cashAndCashEquivalentsAtCarryingValue',
           'totalShareholderEquity', 'netIncome', 'shortLongTermDebtTotal']:
    df[col] = df[col].astype('float')

df['debtToEquity'] = df['totalLiabilities'] / df['totalShareholderEquity']
df['roe'] = df['netIncome'] / df['totalShareholderEquity']
df['ev/ebitda'] = (df.commonStockSharesOutstanding + df.shortLongTermDebtTotal - df.cashAndCashEquivalentsAtCarryingValue) / df.ebitda

temp_date = pd.Series(df.index).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
temp_date.index = df.index
df['date'] = temp_date
df['year'] = df['date'].apply(lambda x: x.year)
df['quarter'] = df['date'].apply(lambda x: x.month / 3)

temp_date1 = pd.Series(earnings_df.index).apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
temp_date1.index = earnings_df.index
earnings_df['date'] = temp_date1
earnings_df['year'] = earnings_df['date'].apply(lambda x: x.year)
earnings_df['quarter'] = earnings_df['date'].apply(lambda x: (x.month//3) +1)


print(df)

                        ebitda     netIncome  \
fiscalDateEnding                               
2018-06-30        1.679500e+10  1.151900e+10   
2018-09-30        2.004300e+10  1.412500e+10   
2018-12-31        2.819100e+10  1.996500e+10   
2019-03-31        1.784300e+10  1.156100e+10   
2019-06-30        1.571000e+10  1.004400e+10   
2019-09-30        2.011600e+10  1.368600e+10   
2019-12-31        2.951900e+10  2.223600e+10   
2020-03-31        1.667800e+10  1.124900e+10   
2020-06-30        1.658600e+10  1.125300e+10   
2020-09-30        1.823700e+10  1.267300e+10   
2020-12-31        3.688300e+10  2.875500e+10   
2021-03-31        3.147800e+10  2.363000e+10   
2021-06-30        2.786600e+10  2.174400e+10   
2021-09-30        2.690900e+10  2.055100e+10   
2021-12-31        4.463200e+10  3.463000e+10   
2022-03-31        3.356700e+10  2.501000e+10   
2022-06-30        2.659000e+10  1.944200e+10   
2022-09-30        2.834900e+10  2.072100e+10   
2022-12-31        3.954200e+10  2.999800

In [49]:
earnings_df

,reportedEPS,date,year,quarter
reportedDate,,,,
2019-01-29,1.05,2019-01-29,2019,1
2019-04-30,0.62,2019-04-30,2019,2
2019-07-30,0.55,2019-07-30,2019,3
2019-10-30,0.76,2019-10-30,2019,4
2020-01-28,1.25,2020-01-28,2020,1
2020-04-30,0.64,2020-04-30,2020,2
2020-07-30,0.65,2020-07-30,2020,3
2020-10-29,0.73,2020-10-29,2020,4
2021-01-27,1.68,2021-01-27,2021,1


In [50]:
# def searchPrice(year, month, df):
#     date = 31
#     while True:
#         cur = df[(df['Date'].dt.day == date) & (df['Date'].dt.month == month) & (df['Date'].dt.year == year)]
#         if len(cur) == 0: 
#             date -= 1
#         else:
#             return cur.Close_x.values

In [51]:
# df['price'] = df['date'].apply(lambda x: searchPrice(x.year, x.month, applep)[0] )


In [52]:
def getData(date, var ,df):
    quarter = (int(date.month - 1) // 3) + 1
    year = date.year
    return df[(df.quarter == quarter) & (df.year == year)][var].values

In [53]:
applep_new = applep.copy()
applep_new['roe'] = applep_new['Date'].apply(lambda x: getData(x,'roe', df)[0])
applep_new['debtToEquity'] = applep_new['Date'].apply(lambda x: getData(x,'debtToEquity', df)[0])
applep_new['ev/ebitda']  = applep_new['Date'].apply(lambda x: getData(x,'ev/ebitda', df)[0])
applep_new['eps'] = applep_new['Date'].apply(lambda x: getData(x,'reportedEPS', earnings_df)[0])
applep_new['dateStr'] = applep_new['Date'].dt.strftime('%Y-%m-%d')

In [54]:
applep_new

,Date,High,Low,Close_x,Close_y,roe,debtToEquity,ev/ebitda,eps,dateStr
0,2019-01-01 00:00:00-05:00,38.271110,34.211504,35.640194,2549.689941,0.109210,2.230663,4.457843,1.05,2019-01-01
1,2019-01-08 00:00:00-05:00,37.230311,35.782346,36.138916,2582.610107,0.109210,2.230663,4.457843,1.05,2019-01-08
2,2019-01-15 00:00:00-05:00,38.037402,36.150950,37.782021,2670.709961,0.109210,2.230663,4.457843,1.05,2019-01-15
3,2019-01-22 00:00:00-05:00,38.097649,36.548492,37.656754,2643.850098,0.109210,2.230663,4.457843,1.05,2019-01-22
4,2019-01-29 00:00:00-05:00,41.357375,37.129121,41.258595,2724.870117,0.109210,2.230663,4.457843,1.05,2019-01-29
...,...,...,...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,150.689917,140.136039,146.406464,3998.840088,0.528813,5.112557,2.692085,1.29,2022-11-29
205,2022-12-06 00:00:00-05:00,147.075443,139.786569,144.269730,3990.560059,0.528813,5.112557,2.692085,1.29,2022-12-06
206,2022-12-13 00:00:00-05:00,149.741373,131.119811,132.168198,3817.659912,0.528813,5.112557,2.692085,1.29,2022-12-13
207,2022-12-20 00:00:00-05:00,136.601432,129.442364,131.658981,3844.820068,0.528813,5.112557,2.692085,1.29,2022-12-20


In [55]:
twitter = pd.read_csv('twitterComments.csv')

In [56]:
reddit = pd.read_csv('reddit_sentiment_scores.csv')

In [57]:
reddit.columns

Index(['Week Start', 'Week End', 'Sentiment Score', 'Weighted Sentiment Score',
       'Weighted Sentiment'],
      dtype='object')

In [58]:
applep_reddit = applep_new.merge(reddit[['Week Start', 'Weighted Sentiment Score']], how='left',
                                 left_on='dateStr', right_on='Week Start')
applep_reddit.drop('Week Start', axis=1, inplace=True)
applep_reddit['Weighted Sentiment Score'] = applep_reddit['Weighted Sentiment Score'].fillna(0)
applep_reddit = applep_reddit.rename(columns={'Weighted Sentiment Score': 'reddit_sentiment'})

In [59]:
apple_twitter = applep_reddit.merge(twitter[['Week Start', 'Twitter_Weighted_Sentiment_Score' ]], how='left', 
                                    left_on='dateStr',right_on='Week Start')
apple_twitter.drop('Week Start', axis=1, inplace=True)
apple_twitter = apple_twitter.rename(columns={'Twitter_Weighted_Sentiment_Score': 'twitter_sentiment'})

In [60]:
apple_twitter

,Date,High,Low,Close_x,Close_y,roe,debtToEquity,ev/ebitda,eps,dateStr,reddit_sentiment,twitter_sentiment
0,2019-01-01 00:00:00-05:00,38.271110,34.211504,35.640194,2549.689941,0.109210,2.230663,4.457843,1.05,2019-01-01,0.000000,0.088160
1,2019-01-08 00:00:00-05:00,37.230311,35.782346,36.138916,2582.610107,0.109210,2.230663,4.457843,1.05,2019-01-08,0.000000,0.195576
2,2019-01-15 00:00:00-05:00,38.037402,36.150950,37.782021,2670.709961,0.109210,2.230663,4.457843,1.05,2019-01-15,0.000000,0.272402
3,2019-01-22 00:00:00-05:00,38.097649,36.548492,37.656754,2643.850098,0.109210,2.230663,4.457843,1.05,2019-01-22,0.000000,0.141259
4,2019-01-29 00:00:00-05:00,41.357375,37.129121,41.258595,2724.870117,0.109210,2.230663,4.457843,1.05,2019-01-29,0.000000,0.137917
...,...,...,...,...,...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,150.689917,140.136039,146.406464,3998.840088,0.528813,5.112557,2.692085,1.29,2022-11-29,0.152096,0.064787
205,2022-12-06 00:00:00-05:00,147.075443,139.786569,144.269730,3990.560059,0.528813,5.112557,2.692085,1.29,2022-12-06,0.158785,0.141762
206,2022-12-13 00:00:00-05:00,149.741373,131.119811,132.168198,3817.659912,0.528813,5.112557,2.692085,1.29,2022-12-13,0.000000,0.124669
207,2022-12-20 00:00:00-05:00,136.601432,129.442364,131.658981,3844.820068,0.528813,5.112557,2.692085,1.29,2022-12-20,0.000000,-0.010471


In [61]:
reddit

,Week Start,Week End,Sentiment Score,Weighted Sentiment Score,Weighted Sentiment
0,2019-08-06,2019-08-12,0.113333,0.113333,positive
1,2019-09-10,2019-09-16,0.000000,0.000000,neutral
2,2020-01-14,2020-01-20,0.292950,0.292950,positive
3,2020-03-31,2020-04-06,0.267850,0.267850,positive
4,2021-01-05,2021-01-11,0.416567,0.416567,positive
...,...,...,...,...,...
88,2022-10-18,2022-10-24,0.337939,0.313955,positive
89,2022-10-25,2022-10-31,0.091461,0.090577,positive
90,2022-11-29,2022-12-05,0.141120,0.152096,positive
91,2022-12-06,2022-12-12,0.251790,0.158785,positive


In [62]:
twitter

,Unnamed: 0,Week Start,Week End,Twitter_Sentiment_Score,Twitter_Weighted_Sentiment_Score,weightedSentiment
0,0,2019-01-01,2019-01-07,0.163786,0.088160,positive
1,1,2019-01-08,2019-01-14,0.273208,0.195576,positive
2,2,2019-01-15,2019-01-21,0.302371,0.272402,positive
3,3,2019-01-22,2019-01-28,0.273153,0.141259,positive
4,4,2019-01-29,2019-02-04,0.287886,0.137917,positive
...,...,...,...,...,...,...
204,204,2022-11-29,2022-12-05,0.189747,0.064787,positive
205,205,2022-12-06,2022-12-12,0.221500,0.141762,positive
206,206,2022-12-13,2022-12-19,0.181143,0.124669,positive
207,207,2022-12-20,2022-12-26,0.179542,-0.010471,neutral


In [63]:
applep_new

,Date,High,Low,Close_x,Close_y,roe,debtToEquity,ev/ebitda,eps,dateStr
0,2019-01-01 00:00:00-05:00,38.271110,34.211504,35.640194,2549.689941,0.109210,2.230663,4.457843,1.05,2019-01-01
1,2019-01-08 00:00:00-05:00,37.230311,35.782346,36.138916,2582.610107,0.109210,2.230663,4.457843,1.05,2019-01-08
2,2019-01-15 00:00:00-05:00,38.037402,36.150950,37.782021,2670.709961,0.109210,2.230663,4.457843,1.05,2019-01-15
3,2019-01-22 00:00:00-05:00,38.097649,36.548492,37.656754,2643.850098,0.109210,2.230663,4.457843,1.05,2019-01-22
4,2019-01-29 00:00:00-05:00,41.357375,37.129121,41.258595,2724.870117,0.109210,2.230663,4.457843,1.05,2019-01-29
...,...,...,...,...,...,...,...,...,...,...
204,2022-11-29 00:00:00-05:00,150.689917,140.136039,146.406464,3998.840088,0.528813,5.112557,2.692085,1.29,2022-11-29
205,2022-12-06 00:00:00-05:00,147.075443,139.786569,144.269730,3990.560059,0.528813,5.112557,2.692085,1.29,2022-12-06
206,2022-12-13 00:00:00-05:00,149.741373,131.119811,132.168198,3817.659912,0.528813,5.112557,2.692085,1.29,2022-12-13
207,2022-12-20 00:00:00-05:00,136.601432,129.442364,131.658981,3844.820068,0.528813,5.112557,2.692085,1.29,2022-12-20


In [64]:
def series_to_supervised(data, target, quarterly, n_in=1, n_out=1, dropnan=True):    
    df = data.loc[:, list(pd.Series(data.columns).apply(lambda x: x not in  quarterly)) ].copy()
    n_vars = df.shape[1]
    df_quarterly = data.loc[:, list(pd.Series(data.columns).apply(lambda x: x in  quarterly))].copy()
    
    quarterly_dict= {}
    for col in quarterly:
        quarterly_dict[col] = []
        
    for i in range(len(df_quarterly)):
        for col, arr in quarterly_dict.items():
            temp = 0
            if i < n_in - 1:
                arr.append(None)
            else:
                
                for j in range(n_in):
                    temp += float(df_quarterly.iloc[i - j][col])       
                arr.append(temp / n_in)

    for col in quarterly:
        df_quarterly[col] = quarterly_dict[col]
        
    
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        if i == 1: 
          names += [ df.columns[j] + '(t)' for j in range(n_vars)]
        else:
          names += [ df.columns[j] +('(t-%d)' % (i-1)) for j in range(n_vars)]
    
    cols.append(df_quarterly.shift(n_in))
    names += list(df_quarterly.columns)
    
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        names += [ target + ('%d(t+%d)' % (1, i+1))]
        

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [65]:
applepen = apple_twitter.copy().drop(columns=['Date', 'dateStr'])
data = series_to_supervised(applepen, 'Close_x',['roe', 'debtToEquity', 'ev/ebitda', 'eps'], 5, 1)
data

,High(t-4),Low(t-4),Close_x(t-4),Close_y(t-4),reddit_sentiment(t-4),twitter_sentiment(t-4),High(t-3),Low(t-3),Close_x(t-3),Close_y(t-3),...,Low(t),Close_x(t),Close_y(t),reddit_sentiment(t),twitter_sentiment(t),roe,debtToEquity,ev/ebitda,eps,Close_x1(t+1)
9,41.357375,37.129121,41.258595,2724.870117,0.000000,0.137917,42.299388,40.576765,40.820099,2709.800049,...,41.793638,42.548553,2792.810059,0.000000,0.345201,0.109210,2.230663,4.457843,1.050,43.286526
10,42.299388,40.576765,40.820099,2709.800049,0.000000,0.262374,41.733154,40.983082,41.234718,2775.600098,...,41.012109,43.286526,2783.300049,0.000000,0.213995,0.109210,2.230663,4.457843,1.050,45.493206
11,41.733154,40.983082,41.234718,2775.600098,0.000000,0.291613,42.553395,41.009697,42.156582,2796.110107,...,43.400255,45.493206,2832.939941,0.000000,0.283504,0.109210,2.230663,4.457843,1.050,45.667412
12,42.553395,41.009697,42.156582,2796.110107,0.000000,0.280743,43.008275,41.793638,42.548553,2792.810059,...,44.697152,45.667412,2798.360107,0.000000,0.259652,0.109210,2.230663,4.457843,1.050,46.272305
13,43.008275,41.793638,42.548553,2792.810059,0.000000,0.345201,43.339757,41.012109,43.286526,2783.300049,...,44.660854,46.272305,2867.189941,0.000000,0.213035,0.109210,2.230663,4.457843,1.050,48.416069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,156.999444,143.671940,152.852661,3871.979980,0.090577,0.226419,154.955949,133.952921,138.478485,3806.800049,...,143.161422,144.000137,3963.939941,0.000000,0.086957,0.504836,5.282353,3.744930,1.272,146.406464
205,154.955949,133.952921,138.478485,3806.800049,0.000000,0.140308,150.050891,134.384808,148.053940,3957.250000,...,140.136039,146.406464,3998.840088,0.152096,0.064787,0.528813,5.112557,2.692085,1.290,144.269730
206,150.050891,134.384808,148.053940,3957.250000,0.000000,0.239292,153.355842,145.927183,147.784348,3949.939941,...,139.786569,144.269730,3990.560059,0.158785,0.141762,0.528813,5.112557,2.692085,1.290,132.168198
207,153.355842,145.927183,147.784348,3949.939941,0.000000,0.125853,151.598536,143.161422,144.000137,3963.939941,...,131.119811,132.168198,3817.659912,0.000000,0.124669,0.528813,5.112557,2.692085,1.290,131.658981


In [66]:
data.to_csv('training.csv')

In [67]:
values = data.values

In [68]:
values[0]

array([4.13573751e+01, 3.71291213e+01, 4.12585945e+01, 2.72487012e+03,
       0.00000000e+00, 1.37917355e-01, 4.22993884e+01, 4.05767649e+01,
       4.08200989e+01, 2.70980005e+03, 0.00000000e+00, 2.62373716e-01,
       4.17331541e+01, 4.09830822e+01, 4.12347183e+01, 2.77560010e+03,
       0.00000000e+00, 2.91612844e-01, 4.25533951e+01, 4.10096968e+01,
       4.21565819e+01, 2.79611011e+03, 0.00000000e+00, 2.80742640e-01,
       4.30082748e+01, 4.17936378e+01, 4.25485535e+01, 2.79281006e+03,
       0.00000000e+00, 3.45200998e-01, 1.09210278e-01, 2.23066314e+00,
       4.45784252e+00, 1.05000000e+00, 4.32865257e+01])

In [69]:
trform_list = []

In [70]:
def minmaxscaler(row, dataset, in_size, out_size, num_var):
  trform_dict = {}
  res = np.zeros(len(row))
  for i in range(num_var): 
    cur_vars = row[[x for x in range(i, in_size * num_var, num_var)]]
    rowmaxmin = cur_vars.max() - cur_vars.min()
    rowmin = cur_vars.min()
    trform_dict[i] = {}
    trform_dict[i]['maxmin'] = rowmaxmin
    trform_dict[i]['min'] = rowmin
    res[[x for x in range(i, in_size * num_var, num_var)]] = [ (x- rowmin) / (rowmaxmin) for x in cur_vars]

    if (i == 0):
      res[-out_size:] = [ (x- rowmin) / (rowmaxmin) for x in row[-out_size:]]
  trform_list.append(trform_dict)
  return res

In [71]:
trform = np.array(list(map(lambda x: minmaxscaler(x, 'test',5,2,2), values)))

In [72]:
trform[0]

array([0.97772986, 0.01357609, 0.97539459, 1.        , 0.        ,
       0.        , 1.        , 0.0148414 , 0.96502811, 0.99446916,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.02482305, 1.02333692])